In [1]:
!asd >& /dev/null
!pgrep -x asd >/dev/null && echo "Aerospike database is running!" || echo "**Aerospike database is not running!**"

Aerospike database is running!


# Aerospike Basic Operations

Basic CRUD (Create, Read, Update, and Delete) operations in Aerospike, and how multiple operations on a record are performed in a single request.
<br>
This notebook requires Aerospike datbase running on localhost and that python and the Aerospike python client have been installed (`pip install aerospike`). Visit [Aerospike notebooks repo](https://github.com/aerospike-examples/interactive-notebooks) for additional details and the docker container.

In [2]:
import aerospike
import sys
config = {
    'hosts': [('127.0.0.1', 3000)]
}

try:
    client = aerospike.client(config).connect()
except:
    import sys
    print("failed to connect to the cluster with", config['hosts'])
    sys.exit(1)
print('Client initialized and connected to DB')

Client initialized and connected to DB


## Understanding Records in Aerospike
Data in Aerospike consists of records. A record belongs to a namespace (equivalent to a database) and optionally to a set (equivalent to a table). A record has multiple bins (or fields), which are named, strongly-typed containers that hold both atomic (string, integer, bytes) and complex (map, list) data types.

A record has two metadata values:
- generation: the number of times it has been modified
- ttl (time to live): seconds remaining until record expiration (default = 0; never expire)

Expired records are garbage-collected by the database. On a write or touch operations, a record's ttl is updated based on the specified policy.

### Record Structure
The following code cell illustrates the record structure.

Note:
- key: (namespace, set, user_key, digest), digest is computed from the first three elements.
- metadata: {'gen': generation, 'ttl': ttl}
- bins: key-value pairs of data bins dictionary

And also:
- digest (the bytearray in output) is the actual record unique identifier.
- user_key input to produce digest may or may not be stored with the record, and is governed by the key policy.


In [24]:
namespace = 'test'
demoset = 'demo'
user_key = 'foo'
meta = {'ttl': 0}
bins = {'name':'John Doe', 'age': 14, 'gpa': 4.3 }
policy = {'key': aerospike.POLICY_KEY_SEND}

try:
    client.put((namespace, demoset, user_key), bins, meta, policy)
except:
    print('failed to put record')
    sys.exit(1)
print('Successfully wrote the record')

try:
    (key, metadata, bins) = client.get((namespace, demoset, user_key), policy)
except:
    print('failed to get record')
    sys.exit(1)

print('Successfully read the record')
print('Key: ', key)
print('Metadata: ', metadata)
print('Bins: ', bins)

Successfully wrote the record
Successfully read the record
Key:  ('test', 'demo', 'foo', bytearray(b'\xf5~\xc1\x835\xf7\x10\x0c\x04X\xf8\xa6D\xbc\xbcvm\x93G\x1e'))
Metadata:  {'ttl': 2592000, 'gen': 1}
Bins:  {'name': 'John Doe', 'age': 14, 'gpa': 4.3}


## Writing Records
The Python client's "put" operations is used to write data to the Aerospike cluster.

### Defining the Key
As described above, the key tuple serves as the record's unique identifier.

Below we define a key tuple in the set "characters" with the user key "bender" in namespace "test".

In [4]:
key = ('test', 'characters', 'bender')

In [5]:
# The record data to write to the cluster
bins = {
  'name': 'Bender',
  'serialnum': 2716057,
  'lastsentence': {
    'BBS': "Well, we're boned",
    'TBwaBB': 'I love you, meatbags!',
    'BG': 'Whip harder, Professor!',
    'ltWGY': 'Into the breach, meatbags. Or not, whatever'},
  'composition': [ "40% zinc", "40% titanium", "30% iron", "40% dolomite" ],
  'apartment': bytearray(b'\x24'),
  'quote_cnt': 47
}

### Storing the Record
Store the record in the database with put.

In [6]:
try:
    client.put(key, bins)
except:
    print('failed to put record')

In [7]:
try:
    (key, metadata, bins) = client.get(key, policy)
except:
    print('failed')
    sys.exit(1)
print('Bins', bins)

Bins {'name': 'Bender', 'serialnum': 2716057, 'lastsentence': {'BBS': "Well, we're boned", 'BG': 'Whip harder, Professor!', 'TBwaBB': 'I love you, meatbags!', 'ltWGY': 'Into the breach, meatbags. Or not, whatever'}, 'composition': ['40% zinc', '40% titanium', '30% iron', '40% dolomite'], 'apartment': b'$', 'quote_cnt': 47}


### Appending, Prepending and Incrementing a Bin
The following APIs are used to prepend and append string bins, and increment integer bins:

In [8]:
try:
    client.prepend(key, 'name', 'DR')
    client.increment(key, 'quote_cnt', 3)
except:
    print('failed to get record')
    sys.exit(1)
    
try:
    (key, metadata, bins) = client.get(key, policy)
except:
    print('failed')
    sys.exit(1)
print(bins)

{'name': 'DRBender', 'serialnum': 2716057, 'lastsentence': {'BBS': "Well, we're boned", 'BG': 'Whip harder, Professor!', 'TBwaBB': 'I love you, meatbags!', 'ltWGY': 'Into the breach, meatbags. Or not, whatever'}, 'composition': ['40% zinc', '40% titanium', '30% iron', '40% dolomite'], 'apartment': b'$', 'quote_cnt': 50}


## Reading Records
There are multiple ways to read records from Aerospike database, "get" being the simplest one.
You will need the key as the record unique identifier as discussed above. Note the record was written above.

It returns:
- key — The key tuple of the record that was read.
- meta — The dict containing the record metadata gen and ttl fields.
- bins — The dict containing the bins of the record.

Meta and bins are None if the record is not found.

In [16]:
key = ('test', 'demo', 'foo')
try:
    (key, meta, bins) = client.get(key, policy)
except:
    print('failed to get record')
    sys.exit(1)

In [17]:
print(key, meta, bins)

('test', 'demo', 'foo', bytearray(b'\xf5~\xc1\x835\xf7\x10\x0c\x04X\xf8\xa6D\xbc\xbcvm\x93G\x1e')) {'ttl': 2591995, 'gen': 3} {'name': 'John Doe', 'age': 14, 'gpa': 4.3}


### Projecting the Bins of a Record
It is possible to project or read specific bins of a record. The following example illustrates this. Note that the second argument is a tuple of bin names to project.

In [28]:
try:
    (key, meta, bins) = client.select(key, ('age', 'name'))
except:
    print('failed')
print(bins)

{'age': 14, 'name': 'John Doe'}


In [29]:
key

('test',
 'demo',
 'foo',
 bytearray(b'\xf5~\xc1\x835\xf7\x10\x0c\x04X\xf8\xa6D\xbc\xbcvm\x93G\x1e'))